In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage, stats
from sklearn.feature_selection import mutual_info_regression
from sklearn.neighbors import NearestNeighbors
import scipy.spatial.distance as dist
from scipy.signal import correlate2d
import warnings
warnings.filterwarnings('ignore')

class SpatioTemporalInfoAnalyzer:
    """
    Comprehensive toolkit for quantifying information content in evolving 2D fields
    """
    
    def __init__(self, field_data, dx=1.0, dt=1.0):
        """
        field_data: 3D array (time, y, x) representing the evolving 2D field
        dx, dt: spatial and temporal resolution
        """
        self.data = np.array(field_data)
        self.T, self.ny, self.nx = self.data.shape
        self.dx = dx
        self.dt = dt
        
    def differential_entropy_kde(self, field_slice, bandwidth=None):
        """
        Estimate differential entropy using kernel density estimation
        """
        field_flat = field_slice.flatten()
        # Remove any NaN or inf values
        field_flat = field_flat[np.isfinite(field_flat)]
        
        if len(field_flat) == 0:
            return 0.0
            
        # Estimate PDF using KDE
        try:
            kde = stats.gaussian_kde(field_flat, bw_method=bandwidth)
            # Evaluate KDE on a grid
            x_grid = np.linspace(field_flat.min(), field_flat.max(), 100)
            pdf_vals = kde(x_grid)
            pdf_vals = pdf_vals[pdf_vals > 1e-10]  # Avoid log(0)
            
            # Numerical integration for entropy
            dx_grid = x_grid[1] - x_grid[0] if len(x_grid) > 1 else 1.0
            entropy = -np.sum(pdf_vals * np.log(pdf_vals)) * dx_grid
            return entropy
        except:
            return 0.0
    
    def spatial_mutual_information(self, t1, t2, lag_x=1, lag_y=1, bins=50):
        """
        Calculate mutual information between spatially separated regions
        """
        field1 = self.data[t1]
        field2 = self.data[t2]
        
        # Create spatially lagged versions
        region1 = field1[:-lag_y, :-lag_x].flatten()
        region2 = field2[lag_y:, lag_x:].flatten()
        
        # Remove invalid values
        valid_mask = np.isfinite(region1) & np.isfinite(region2)
        region1 = region1[valid_mask]
        region2 = region2[valid_mask]
        
        if len(region1) < 10:
            return 0.0
        
        # Calculate mutual information using binning
        try:
            hist_2d, x_edges, y_edges = np.histogram2d(region1, region2, bins=bins)
            hist_2d = hist_2d + 1e-10  # Avoid log(0)
            
            # Normalize to get probabilities
            p_xy = hist_2d / np.sum(hist_2d)
            p_x = np.sum(p_xy, axis=1)
            p_y = np.sum(p_xy, axis=0)
            
            # Calculate MI
            mi = 0.0
            for i in range(len(p_x)):
                for j in range(len(p_y)):
                    if p_xy[i,j] > 0 and p_x[i] > 0 and p_y[j] > 0:
                        mi += p_xy[i,j] * np.log(p_xy[i,j] / (p_x[i] * p_y[j]))
            
            return mi
        except:
            return 0.0
    
    def transfer_entropy_estimator(self, source_ts, target_ts, lag=1, bins=10):
        """
        Estimate transfer entropy between two time series
        TE(X->Y) = I(Y_t+1; X_t | Y_t)
        """
        if len(source_ts) != len(target_ts) or len(source_ts) < lag + 2:
            return 0.0
        
        # Prepare variables for TE calculation
        y_future = target_ts[lag+1:]
        x_past = source_ts[lag:-1]
        y_past = target_ts[lag:-1]
        
        # Remove invalid values
        valid_mask = np.isfinite(y_future) & np.isfinite(x_past) & np.isfinite(y_past)
        y_future = y_future[valid_mask]
        x_past = x_past[valid_mask]
        y_past = y_past[valid_mask]
        
        if len(y_future) < 10:
            return 0.0
        
        try:
            # Calculate conditional mutual information
            # TE = I(Y_t+1; X_t | Y_t) = H(Y_t+1 | Y_t) - H(Y_t+1 | X_t, Y_t)
            
            # Bin the data
            y_fut_binned = np.digitize(y_future, np.linspace(y_future.min(), y_future.max(), bins))
            x_past_binned = np.digitize(x_past, np.linspace(x_past.min(), x_past.max(), bins))
            y_past_binned = np.digitize(y_past, np.linspace(y_past.min(), y_past.max(), bins))
            
            # Calculate joint and marginal entropies
            def joint_entropy_2d(x, y):
                hist, _, _ = np.histogram2d(x, y, bins=bins)
                hist = hist + 1e-10
                p = hist / np.sum(hist)
                return -np.sum(p * np.log(p))
            
            def joint_entropy_3d(x, y, z):
                # Simplified 3D entropy calculation
                combined = x * bins**2 + y * bins + z
                unique, counts = np.unique(combined, return_counts=True)
                p = counts / np.sum(counts)
                p = p + 1e-10
                return -np.sum(p * np.log(p))
            
            h_y_fut_y_past = joint_entropy_2d(y_future, y_past)
            h_y_past = -np.sum((np.bincount(y_past_binned) / len(y_past_binned)) * 
                              np.log(np.bincount(y_past_binned) / len(y_past_binned) + 1e-10))
            
            h_y_fut_x_past_y_past = joint_entropy_3d(y_fut_binned, x_past_binned, y_past_binned)
            h_x_past_y_past = joint_entropy_2d(x_past, y_past)
            
            # TE = I(Y_fut; X_past | Y_past)
            te = h_y_fut_y_past + h_x_past_y_past - h_y_fut_x_past_y_past - h_y_past
            
            return max(0, te)  # TE should be non-negative
        except:
            return 0.0
    
    def entropy_rate(self, window_size=5):
        """
        Calculate entropy rate - new information per time step
        """
        rates = []
        for t in range(window_size, self.T):
            current_entropy = self.differential_entropy_kde(self.data[t])
            
            # Conditional entropy given past window
            past_window = self.data[t-window_size:t]
            past_info = np.mean([self.differential_entropy_kde(frame) for frame in past_window])
            
            # Rate is new entropy beyond what's predictable from past
            rate = current_entropy - 0.8 * past_info  # 0.8 is a damping factor
            rates.append(max(0, rate))
        
        return np.array(rates)
    
    def information_flow_field(self, lag=1):
        """
        Calculate information flow between neighboring spatial regions
        Returns a 2D field showing local information transfer rates
        """
        flow_field = np.zeros((self.ny-2, self.nx-2))
        
        for i in range(1, self.ny-1):
            for j in range(1, self.nx-1):
                # Extract time series for center pixel and neighbors
                center_ts = self.data[:, i, j]
                
                # Calculate average transfer entropy from neighbors
                neighbors = [
                    self.data[:, i-1, j],    # up
                    self.data[:, i+1, j],    # down
                    self.data[:, i, j-1],    # left
                    self.data[:, i, j+1],    # right
                ]
                
                te_values = []
                for neighbor_ts in neighbors:
                    te = self.transfer_entropy_estimator(neighbor_ts, center_ts, lag)
                    te_values.append(te)
                
                flow_field[i-1, j-1] = np.mean(te_values)
        
        return flow_field
    
    def wavefront_info_velocity(self):
        """
        Estimate the velocity of information propagation
        """
        velocities = []
        
        for t in range(1, self.T-1):
            # Calculate spatial gradients of information
            info_field = np.abs(self.data[t+1] - self.data[t-1]) / (2 * self.dt)
            
            # Estimate wavefront using edge detection
            grad_x = np.gradient(info_field, self.dx, axis=1)
            grad_y = np.gradient(info_field, self.dx, axis=0)
            
            # Information velocity magnitude
            velocity_mag = np.sqrt(grad_x**2 + grad_y**2)
            velocities.append(np.mean(velocity_mag[np.isfinite(velocity_mag)]))
        
        return np.array(velocities)
    
    def analyze_complete(self):
        """
        Perform comprehensive spatio-temporal information analysis
        """
        print("Performing comprehensive spatio-temporal information analysis...")
        
        results = {}
        
        # 1. Temporal entropy evolution
        print("  - Calculating temporal entropy evolution...")
        temporal_entropies = []
        for t in range(self.T):
            entropy = self.differential_entropy_kde(self.data[t])
            temporal_entropies.append(entropy)
        results['temporal_entropy'] = np.array(temporal_entropies)
        
        # 2. Entropy rate
        print("  - Calculating entropy rate...")
        results['entropy_rate'] = self.entropy_rate()
        
        # 3. Spatial mutual information evolution
        print("  - Calculating spatial correlations...")
        spatial_mi = []
        for t in range(self.T-1):
            mi = self.spatial_mutual_information(t, t+1, lag_x=1, lag_y=1)
            spatial_mi.append(mi)
        results['spatial_mi'] = np.array(spatial_mi)
        
        # 4. Information flow field
        print("  - Calculating information flow field...")
        results['flow_field'] = self.information_flow_field()
        
        # 5. Information velocity
        print("  - Estimating information propagation velocity...")
        results['info_velocity'] = self.wavefront_info_velocity()
        
        return results
    
    def plot_analysis(self, results):
        """
        Visualize the information analysis results
        """
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        
        # Original field evolution (show first, middle, last frames)
        times_to_show = [0, self.T//2, self.T-1]
        for idx, t in enumerate(times_to_show):
            ax = axes[0, idx]
            im = ax.imshow(self.data[t], cmap='viridis', aspect='auto')
            ax.set_title(f'Field at t={t}')
            plt.colorbar(im, ax=ax)
        
        # Temporal entropy evolution
        axes[1, 0].plot(results['temporal_entropy'])
        axes[1, 0].set_title('Temporal Entropy Evolution')
        axes[1, 0].set_xlabel('Time')
        axes[1, 0].set_ylabel('Differential Entropy')
        
        # Entropy rate
        axes[1, 1].plot(results['entropy_rate'])
        axes[1, 1].set_title('Information Generation Rate')
        axes[1, 1].set_xlabel('Time')
        axes[1, 1].set_ylabel('Entropy Rate')
        
        # Information flow field
        im = axes[1, 2].imshow(results['flow_field'], cmap='plasma', aspect='auto')
        axes[1, 2].set_title('Information Flow Field')
        plt.colorbar(im, ax=axes[1, 2])
        
        plt.tight_layout()
        plt.show()
        
        # Summary statistics
        print("\n=== SPATIO-TEMPORAL INFORMATION ANALYSIS SUMMARY ===")
        print(f"Total entropy change: {results['temporal_entropy'][-1] - results['temporal_entropy'][0]:.3f}")
        print(f"Average entropy rate: {np.mean(results['entropy_rate']):.3f}")
        print(f"Peak information generation: {np.max(results['entropy_rate']):.3f}")
        print(f"Average spatial MI: {np.mean(results['spatial_mi']):.3f}")
        print(f"Mean information flow: {np.mean(results['flow_field']):.3f}")
        print(f"Average information velocity: {np.mean(results['info_velocity']):.3f}")

# Example usage with simulated data
def generate_example_field():
    """Generate a sample evolving 2D field (wave equation solution)"""
    nx, ny, nt = 50, 50, 100
    dx, dy, dt = 0.1, 0.1, 0.01
    
    x = np.linspace(0, 5, nx)
    y = np.linspace(0, 5, ny)
    t = np.linspace(0, 1, nt)
    
    X, Y = np.meshgrid(x, y)
    field_data = np.zeros((nt, ny, nx))
    
    # Simulate a wave propagating with some nonlinearity
    for i, time in enumerate(t):
        # Multiple wave sources with different frequencies
        wave1 = np.sin(2*np.pi*(X - 2*time)) * np.exp(-((X-2.5)**2 + (Y-2.5)**2)/2)
        wave2 = 0.5*np.sin(3*np.pi*(Y - 1.5*time)) * np.exp(-((X-1)**2 + (Y-4)**2)/1)
        
        # Add some nonlinear interaction
        field_data[i] = wave1 + wave2 + 0.1*wave1*wave2
        
        # Add small amount of noise
        field_data[i] += 0.05 * np.random.randn(ny, nx)
    
    return field_data

if __name__ == "__main__":
    # Generate example data
    print("Generating example 2D evolving field...")
    field_data = generate_example_field()
    
    # Create analyzer
    analyzer = SpatioTemporalInfoAnalyzer(field_data, dx=0.1, dt=0.01)
    
    # Perform analysis
    results = analyzer.analyze_complete()
    
    # Visualize results
    analyzer.plot_analysis(results)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import DBSCAN
import networkx as nx
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

class PersistentHomologyInfoAnalyzer:
    """
    Analyze information flow using persistent homology of evolving 2D fields
    """
    
    def __init__(self, field_data, threshold_method='percentile'):
        """
        field_data: 3D array (time, y, x)
        threshold_method: 'percentile', 'adaptive', or 'fixed'
        """
        self.data = np.array(field_data)
        self.T, self.ny, self.nx = self.data.shape
        self.threshold_method = threshold_method
        
    def compute_sublevel_sets(self, field_slice, thresholds):
        """
        Compute sublevel sets for different threshold values
        Returns binary masks for each threshold
        """
        sublevel_sets = []
        for thresh in thresholds:
            sublevel = field_slice <= thresh
            sublevel_sets.append(sublevel)
        return sublevel_sets
    
    def compute_superlevel_sets(self, field_slice, thresholds):
        """
        Compute superlevel sets for different threshold values
        More relevant for information flow (high activity regions)
        """
        superlevel_sets = []
        for thresh in thresholds:
            superlevel = field_slice >= thresh
            superlevel_sets.append(superlevel)
        return superlevel_sets
    
    def connected_components_2d(self, binary_mask):
        """
        Find connected components in 2D binary mask
        Returns component labels and count
        """
        labeled, num_components = ndimage.label(binary_mask)
        return labeled, num_components
    
    def find_loops_2d(self, binary_mask):
        """
        Simplified loop detection using Euler characteristic
        For 2D: χ = V - E + F, loops create holes (reduce χ)
        """
        labeled, num_components = self.connected_components_2d(binary_mask)
        
        if num_components == 0:
            return 0
        
        # Calculate Euler characteristic for each component
        total_loops = 0
        for comp_id in range(1, num_components + 1):
            component = (labeled == comp_id)
            
            # Count vertices (pixels), edges, and estimate loops
            vertices = np.sum(component)
            if vertices < 4:  # Too small for a loop
                continue
                
            # Estimate edges by counting neighbor connections
            edges = 0
            for i in range(self.ny-1):
                for j in range(self.nx-1):
                    if component[i,j]:
                        # Count connections to neighbors
                        if j < self.nx-1 and component[i,j+1]:
                            edges += 1
                        if i < self.ny-1 and component[i+1,j]:
                            edges += 1
            
            # For 2D grid: χ = 1 - loops (for connected component)
            # Rough approximation: loops ≈ edges - vertices + 1
            component_loops = max(0, edges - vertices + 1)
            total_loops += component_loops
        
        return total_loops
    
    def persistence_diagram(self, field_slice, level_type='super'):
        """
        Compute persistence diagram for a single field slice
        Returns birth-death pairs for H0 (components) and H1 (loops)
        """
        # Determine threshold range
        field_min, field_max = np.min(field_slice), np.max(field_slice)
        
        if self.threshold_method == 'percentile':
            thresholds = np.percentile(field_slice.flatten(), np.linspace(5, 95, 20))
        elif self.threshold_method == 'adaptive':
            mean_val = np.mean(field_slice)
            std_val = np.std(field_slice)
            thresholds = np.linspace(mean_val - 2*std_val, mean_val + 2*std_val, 20)
        else:  # fixed
            thresholds = np.linspace(field_min, field_max, 20)
        
        thresholds = np.sort(thresholds)
        
        # Compute level sets
        if level_type == 'super':
            level_sets = self.compute_superlevel_sets(field_slice, thresholds)
        else:
            level_sets = self.compute_sublevel_sets(field_slice, thresholds)
        
        # Track components and loops across thresholds
        h0_births = []  # Component births
        h0_deaths = []  # Component deaths
        h1_births = []  # Loop births
        h1_deaths = []  # Loop deaths
        
        prev_components = 0
        prev_loops = 0
        component_tracker = {}
        loop_tracker = {}
        
        for i, (thresh, level_set) in enumerate(zip(thresholds, level_sets)):
            # Count components and loops
            _, num_components = self.connected_components_2d(level_set)
            num_loops = self.find_loops_2d(level_set)
            
            # Track component births and deaths
            if num_components > prev_components:
                # New components born
                for _ in range(num_components - prev_components):
                    h0_births.append(thresh)
            elif num_components < prev_components:
                # Components died
                for _ in range(prev_components - num_components):
                    h0_deaths.append(thresh)
            
            # Track loop births and deaths
            if num_loops > prev_loops:
                for _ in range(num_loops - prev_loops):
                    h1_births.append(thresh)
            elif num_loops < prev_loops:
                for _ in range(prev_loops - num_loops):
                    h1_deaths.append(thresh)
            
            prev_components = num_components
            prev_loops = num_loops
        
        # Match births with deaths to create persistence pairs
        h0_pairs = self._match_births_deaths(h0_births, h0_deaths, thresholds[-1])
        h1_pairs = self._match_births_deaths(h1_births, h1_deaths, thresholds[-1])
        
        return {
            'H0': h0_pairs,  # Connected components
            'H1': h1_pairs,  # Loops/holes
            'thresholds': thresholds
        }
    
    def _match_births_deaths(self, births, deaths, max_thresh):
        """
        Match birth and death events to create persistence pairs
        """
        births = sorted(births)
        deaths = sorted(deaths)
        
        pairs = []
        
        # Match deaths with most recent births
        birth_idx = 0
        for death in deaths:
            if birth_idx < len(births):
                pairs.append((births[birth_idx], death))
                birth_idx += 1
        
        # Remaining births persist to infinity (max threshold)
        while birth_idx < len(births):
            pairs.append((births[birth_idx], max_thresh))
            birth_idx += 1
        
        return pairs
    
    def persistence_landscape(self, persistence_pairs, resolution=100):
        """
        Convert persistence diagram to persistence landscape
        More stable representation for statistical analysis
        """
        if not persistence_pairs:
            return np.zeros(resolution), np.linspace(0, 1, resolution)
        
        # Extract birth and death times
        births = [pair[0] for pair in persistence_pairs]
        deaths = [pair[1] for pair in persistence_pairs]
        
        if not births:
            return np.zeros(resolution), np.linspace(0, 1, resolution)
        
        min_val = min(births)
        max_val = max(deaths)
        
        if max_val == min_val:
            return np.zeros(resolution), np.linspace(min_val, min_val + 1, resolution)
        
        x = np.linspace(min_val, max_val, resolution)
        landscape = np.zeros(resolution)
        
        # For each persistence pair, add triangle function
        for birth, death in persistence_pairs:
            persistence = death - birth
            midpoint = (birth + death) / 2
            
            # Create triangle function
            for i, xi in enumerate(x):
                if birth <= xi <= death:
                    if xi <= midpoint:
                        landscape[i] += (xi - birth) / (midpoint - birth) * persistence / 2
                    else:
                        landscape[i] += (death - xi) / (death - midpoint) * persistence / 2
        
        return landscape, x
    
    def bottleneck_distance(self, pairs1, pairs2):
        """
        Compute bottleneck distance between two persistence diagrams
        Simplified version using maximum matching
        """
        if not pairs1 and not pairs2:
            return 0.0
        if not pairs1 or not pairs2:
            # Distance to empty diagram
            all_pairs = pairs1 if pairs1 else pairs2
            return max([abs(death - birth) for birth, death in all_pairs])
        
        # Compute pairwise distances
        distances = []
        for b1, d1 in pairs1:
            min_dist = float('inf')
            for b2, d2 in pairs2:
                dist = max(abs(b1 - b2), abs(d1 - d2))
                min_dist = min(min_dist, dist)
            distances.append(min_dist)
        
        return max(distances) if distances else 0.0
    
    def information_flow_velocity_topological(self):
        """
        Estimate information flow velocity using topological feature tracking
        """
        velocities = []
        
        for t in range(self.T - 1):
            # Get persistence diagrams for consecutive frames
            pd1 = self.persistence_diagram(self.data[t])
            pd2 = self.persistence_diagram(self.data[t + 1])
            
            # Track component centroid movement
            component_velocities = []
            
            # Get component centroids for both frames
            _, components1 = self.connected_components_2d(self.data[t] > np.percentile(self.data[t], 75))
            _, components2 = self.connected_components_2d(self.data[t+1] > np.percentile(self.data[t+1], 75))
            
            centroids1 = []
            for comp_id in range(1, components1 + 1):
                mask = (ndimage.label(self.data[t] > np.percentile(self.data[t], 75))[0] == comp_id)
                if np.sum(mask) > 0:
                    cy, cx = ndimage.center_of_mass(mask)
                    centroids1.append((cx, cy))
            
            centroids2 = []
            for comp_id in range(1, components2 + 1):
                mask = (ndimage.label(self.data[t+1] > np.percentile(self.data[t+1], 75))[0] == comp_id)
                if np.sum(mask) > 0:
                    cy, cx = ndimage.center_of_mass(mask)
                    centroids2.append((cx, cy))
            
            # Match centroids and compute velocities
            if centroids1 and centroids2:
                for c1 in centroids1:
                    min_dist = float('inf')
                    best_match = None
                    for c2 in centroids2:
                        dist = np.sqrt((c1[0] - c2[0])**2 + (c1[1] - c2[1])**2)
                        if dist < min_dist:
                            min_dist = dist
                            best_match = c2
                    
                    if best_match and min_dist < 10:  # Reasonable matching threshold
                        velocity = min_dist  # Distance per time step
                        component_velocities.append(velocity)
            
            avg_velocity = np.mean(component_velocities) if component_velocities else 0
            velocities.append(avg_velocity)
        
        return np.array(velocities)
    
    def topological_complexity_evolution(self):
        """
        Track evolution of topological complexity over time
        """
        h0_complexity = []  # Component complexity
        h1_complexity = []  # Loop complexity
        bottleneck_distances = []
        
        prev_pd = None
        
        for t in range(self.T):
            pd = self.persistence_diagram(self.data[t])
            
            # Complexity measures
            h0_pers = [death - birth for birth, death in pd['H0']]
            h1_pers = [death - birth for birth, death in pd['H1']]
            
            h0_complexity.append(np.sum(h0_pers))
            h1_complexity.append(np.sum(h1_pers))
            
            # Bottleneck distance to previous frame
            if prev_pd is not None:
                bd_h0 = self.bottleneck_distance(prev_pd['H0'], pd['H0'])
                bd_h1 = self.bottleneck_distance(prev_pd['H1'], pd['H1'])
                bottleneck_distances.append(bd_h0 + bd_h1)
            
            prev_pd = pd
        
        return {
            'h0_complexity': np.array(h0_complexity),
            'h1_complexity': np.array(h1_complexity),
            'bottleneck_distances': np.array(bottleneck_distances),
            'total_complexity': np.array(h0_complexity) + np.array(h1_complexity)
        }
    
    def information_creation_annihilation(self):
        """
        Identify regions where topological information is created/destroyed
        """
        creation_map = np.zeros((self.T-1, self.ny, self.nx))
        annihilation_map = np.zeros((self.T-1, self.ny, self.nx))
        
        for t in range(self.T - 1):
            # Get high-activity regions for both frames
            thresh1 = np.percentile(self.data[t], 80)
            thresh2 = np.percentile(self.data[t+1], 80)
            
            active1 = self.data[t] > thresh1
            active2 = self.data[t+1] > thresh2
            
            # Information creation: new active regions
            creation = active2 & ~active1
            creation_map[t] = creation.astype(float)
            
            # Information annihilation: lost active regions
            annihilation = active1 & ~active2
            annihilation_map[t] = annihilation.astype(float)
        
        return creation_map, annihilation_map
    
    def analyze_complete_topological(self):
        """
        Complete topological analysis of information flow
        """
        print("Performing topological information flow analysis...")
        
        results = {}
        
        # 1. Topological complexity evolution
        print("  - Computing topological complexity evolution...")
        results['complexity'] = self.topological_complexity_evolution()
        
        # 2. Information flow velocity
        print("  - Estimating topological flow velocity...")
        results['topo_velocity'] = self.information_flow_velocity_topological()
        
        # 3. Information creation/annihilation maps
        print("  - Mapping information creation/annihilation...")
        creation_map, annihilation_map = self.information_creation_annihilation()
        results['creation_map'] = creation_map
        results['annihilation_map'] = annihilation_map
        
        # 4. Persistence landscapes for representative frames
        print("  - Computing persistence landscapes...")
        representative_frames = [0, self.T//4, self.T//2, 3*self.T//4, self.T-1]
        results['landscapes'] = {}
        
        for frame in representative_frames:
            pd = self.persistence_diagram(self.data[frame])
            landscape_h0, x_h0 = self.persistence_landscape(pd['H0'])
            landscape_h1, x_h1 = self.persistence_landscape(pd['H1'])
            
            results['landscapes'][frame] = {
                'H0': (landscape_h0, x_h0),
                'H1': (landscape_h1, x_h1)
            }
        
        return results
    
    def plot_topological_analysis(self, results):
        """
        Visualize topological analysis results
        """
        fig = plt.figure(figsize=(20, 15))
        
        # 1. Original field evolution
        ax1 = plt.subplot(3, 4, 1)
        im1 = ax1.imshow(self.data[0], cmap='viridis')
        ax1.set_title('Initial Field')
        plt.colorbar(im1, ax=ax1)
        
        ax2 = plt.subplot(3, 4, 2)
        im2 = ax2.imshow(self.data[self.T//2], cmap='viridis')
        ax2.set_title('Mid-evolution Field')
        plt.colorbar(im2, ax=ax2)
        
        ax3 = plt.subplot(3, 4, 3)
        im3 = ax3.imshow(self.data[-1], cmap='viridis')
        ax3.set_title('Final Field')
        plt.colorbar(im3, ax=ax3)
        
        # 2. Topological complexity evolution
        ax4 = plt.subplot(3, 4, 4)
        ax4.plot(results['complexity']['h0_complexity'], label='H0 (Components)', linewidth=2)
        ax4.plot(results['complexity']['h1_complexity'], label='H1 (Loops)', linewidth=2)
        ax4.plot(results['complexity']['total_complexity'], label='Total', linewidth=2, linestyle='--')
        ax4.set_title('Topological Complexity Evolution')
        ax4.set_xlabel('Time')
        ax4.set_ylabel('Persistence Sum')
        ax4.legend()
        ax4.grid(True)
        
        # 3. Bottleneck distances (topological change rate)
        ax5 = plt.subplot(3, 4, 5)
        if len(results['complexity']['bottleneck_distances']) > 0:
            ax5.plot(results['complexity']['bottleneck_distances'], 'r-', linewidth=2)
        ax5.set_title('Topological Change Rate')
        ax5.set_xlabel('Time')
        ax5.set_ylabel('Bottleneck Distance')
        ax5.grid(True)
        
        # 4. Information flow velocity
        ax6 = plt.subplot(3, 4, 6)
        if len(results['topo_velocity']) > 0:
            ax6.plot(results['topo_velocity'], 'g-', linewidth=2)
        ax6.set_title('Topological Flow Velocity')
        ax6.set_xlabel('Time')
        ax6.set_ylabel('Velocity')
        ax6.grid(True)
        
        # 5. Information creation map (average)
        ax7 = plt.subplot(3, 4, 7)
        creation_avg = np.mean(results['creation_map'], axis=0)
        im7 = ax7.imshow(creation_avg, cmap='Reds')
        ax7.set_title('Information Creation Map')
        plt.colorbar(im7, ax=ax7)
        
        # 6. Information annihilation map (average)
        ax8 = plt.subplot(3, 4, 8)
        annihilation_avg = np.mean(results['annihilation_map'], axis=0)
        im8 = ax8.imshow(annihilation_avg, cmap='Blues')
        ax8.set_title('Information Annihilation Map')
        plt.colorbar(im8, ax=ax8)
        
        # 7-8. Persistence landscapes
        ax9 = plt.subplot(3, 4, 9)
        ax10 = plt.subplot(3, 4, 10)
        
        colors = ['blue', 'red', 'green', 'orange', 'purple']
        for i, (frame, color) in enumerate(zip(results['landscapes'].keys(), colors)):
            if frame in results['landscapes']:
                landscape_h0, x_h0 = results['landscapes'][frame]['H0']
                if len(landscape_h0) > 0:
                    ax9.plot(x_h0, landscape_h0, color=color, label=f't={frame}', alpha=0.7)
                
                landscape_h1, x_h1 = results['landscapes'][frame]['H1']
                if len(landscape_h1) > 0:
                    ax10.plot(x_h1, landscape_h1, color=color, label=f't={frame}', alpha=0.7)
        
        ax9.set_title('H0 Persistence Landscapes')
        ax9.set_xlabel('Threshold')
        ax9.set_ylabel('Landscape Value')
        ax9.legend()
        ax9.grid(True)
        
        ax10.set_title('H1 Persistence Landscapes')
        ax10.set_xlabel('Threshold')
        ax10.set_ylabel('Landscape Value')
        ax10.legend()
        ax10.grid(True)
        
        plt.tight_layout()
        plt.show()
        
        # Summary
        print("\n=== TOPOLOGICAL INFORMATION ANALYSIS SUMMARY ===")
        print(f"Peak topological complexity: {np.max(results['complexity']['total_complexity']):.3f}")
        print(f"Average complexity change rate: {np.mean(results['complexity']['bottleneck_distances']):.3f}")
        print(f"Average topological flow velocity: {np.mean(results['topo_velocity']):.3f}")
        print(f"Total information creation events: {np.sum(results['creation_map']):.0f}")
        print(f"Total information annihilation events: {np.sum(results['annihilation_map']):.0f}")

# Example usage
def generate_complex_field():
    """Generate a field with rich topological structure"""
    nx, ny, nt = 60, 60, 80
    x = np.linspace(0, 6, nx)
    y = np.linspace(0, 6, ny)
    t = np.linspace(0, 2, nt)
    
    X, Y = np.meshgrid(x, y)
    field_data = np.zeros((nt, ny, nx))
    
    for i, time in enumerate(t):
        # Multiple interacting waves creating complex topology
        wave1 = np.sin(2*np.pi*(X - 1.5*time)) * np.exp(-((X-3)**2 + (Y-3)**2)/3)
        wave2 = np.sin(3*np.pi*(Y - 1.2*time)) * np.exp(-((X-1)**2 + (Y-5)**2)/2)
        wave3 = np.cos(1.5*np.pi*((X-2*time)**2 + (Y-1.5*time)**2)) * np.exp(-((X-4)**2 + (Y-2)**2)/4)
        
        # Nonlinear interactions create loops and complex structures
        field = wave1 + wave2 + wave3
        field += 0.3 * wave1 * wave2 + 0.2 * wave2 * wave3
        
        # Add localized "vortices" that create topological features
        vortex1 = np.sin(4*time) * np.exp(-((X-2)**2 + (Y-4)**2)/1.5)
        vortex2 = np.cos(3*time) * np.exp(-((X-5)**2 + (Y-1)**2)/1.5)
        
        field += 0.5 * (vortex1 + vortex2)
        
        # Small noise
        field += 0.1 * np.random.randn(ny, nx)
        
        field_data[i] = field
    
    return field_data

if __name__ == "__main__":
    # Generate example with rich topological structure
    print("Generating complex field with topological features...")
    field_data = generate_complex_field()
    
    # Create analyzer
    analyzer = PersistentHomologyInfoAnalyzer(field_data, threshold_method='percentile')
    
    # Perform analysis
    results = analyzer.analyze_complete_topological()
    
    # Visualize
    analyzer.plot_topological_analysis(results)